## PyTorch Tutorial #03 - Autograd

In [2]:
import torch

### Requires_grad = True

In [4]:
x = torch.randn(3, requires_grad = True)
x

tensor([ 0.0746, -1.6187,  1.3421], requires_grad=True)

In [5]:
# Esta operación de suma tiene asociada una función grad_fn = ADD
y = x + 2
y

tensor([2.0746, 0.3813, 3.3421], grad_fn=<AddBackward0>)

In [7]:
# Esta operación de multiplicación tiene asociada una función grad_fn = MUL
z = y * y * 2
z

tensor([ 8.6083,  0.2908, 22.3398], grad_fn=<MulBackward0>)

In [8]:
# Esta operación promedio tiene asociada una función grad_fn = MEAN
z = z.mean()
z

tensor(10.4129, grad_fn=<MeanBackward0>)

In [9]:
# Esto ejecuta la derivada de z respecto de x. dz/dx
z.backward()

In [10]:
# Una vez aplicado la función BACKWARD, puedo mostrar el gradiente x.
x.grad

tensor([2.7662, 0.5084, 4.4562])

### Requires_grad = False

In [11]:
# Cuando especificamos requires_grad = False, vemos que no tiene el atributo grad_fn.
x = torch.randn(3, requires_grad = False)
x

tensor([-1.1266,  1.1203,  0.2888])

In [12]:
# Cuando aplicamos backward(), vemos que arroja error.
y = x + 2
z = y * y * 2
z = z.mean()
z.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

### Gradiente con más de un elemento (vector gradiente)

In [13]:
# Volvemos a setear True para seguir con más ejemplos.
x = torch.randn(3, requires_grad = True)
x

tensor([ 0.3314, -0.0288,  0.2632], requires_grad=True)

In [14]:
# Si llamo a backward va a producir un error porque funciona solo con un output escala (no con vectores).
y = x + 2
z = y * y * 2
z.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [15]:
# Si, en cambio, creo un vector con las mismas dimensiones y se lo paso al argumento de la función backwards, va a funcionar.
# Lo que está ocurriendo acá es un producto vectorial jacobiano.
v = torch.tensor([0.1, 1.0, 0.001], dtype = torch.float32)
z.backward(v)
x.grad

tensor([0.9326, 7.8848, 0.0091])

### Evitar que PyTorch trackee los gradientes

In [16]:
# Hay 3 métodos:

# 1) x.requires_grad(False)
# 2) x.detach()
# 3) with torch.no_grad():

In [18]:
# 1) x.requires_grad(False)
# Notar que se utilizó el guión bajo para modificar la variable.
x = torch.randn(3, requires_grad = True)
x.requires_grad_(False)
x

tensor([-0.4404, -0.0023,  0.6898])

In [19]:
# 2) x.detach()
x = torch.randn(3, requires_grad = True)
y = x.detach()
y

tensor([-0.1725,  1.3180,  0.2547])

In [21]:
# 3) with torch.no_grad():
x = torch.randn(3, requires_grad = True)
with torch.no_grad():
    y = x + 2
y

tensor([1.0660, 2.8850, 3.6334])

### Se acumulan los gradientes

In [30]:
# Los gradientes, cada vez que llamamos a la función backward, se acumulan.
# Notar cómo van cambiando en el loop for.
weights = torch.ones(4, requires_grad = True)
for epoch in range(3):
    model_output = (weights*3).sum()

    model_output.backward()

    print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


In [33]:
# Para evitar esto, es necesario, luego de aplicar backward, setear los gradientes a cero.
weights = torch.ones(4, requires_grad = True)
for epoch in range(3):
    model_output = (weights*3).sum()

    model_output.backward()

    print(weights.grad)

    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [39]:
# Cuando utilicemos una función de costo de PyTorch, hay que reiniciar los gradientes.

weights = torch.ones(4, requires_grad = True)

optimizer = torch.optim.SGD([weights], lr = 0.01)
optimizer.step()
optimizer.zero_grad()